In [11]:
import pandas as pd

In [13]:
df1 = pd.read_csv('./../../data/goodreads_data.csv')
df1.head(5)

,title,author,price,description,author_description,genres,n_ratings,n_reviews,ratings,pages_format,publication_info,literary_awards,original_title,series,characters,format,published,isbn,language,setting
0,The Weed That Strings the Hangman's Bag,Alan Bradley,4.99,From Dagger Award-winning and internationally ...,Librarian Note: There is more than one author ...,"['Mystery', 'Fiction', 'Historical Fiction', '...",55692.0,5756.0,"[395.0, 1540.0, 12237.0, 25714.0, 15806.0]","364 pages, Hardcover","First published March 9, 2010",['Goodreads Choice Award Nominee for Mystery &...,The Weed That Strings the Hangman's Bag,['Flavia de Luce '],"['Flavia de Luce', 'Ophelia de Luce', 'Daphne ...","364 pages, Hardcover","March 9, 2010 by Delacorte Press",9780385342315 (ISBN10: 0385342314),English,NaN
1,A Good Man in Africa,William Boyd,11.99,"In the small African republic of Kinjanja, Bri...","Note: William^^BoydOf Scottish descent, Boyd w...","['Fiction', 'Africa', 'Humor', 'Novels', 'Hist...",5018.0,298.0,"[90.0, 257.0, 1222.0, 2184.0, 1265.0]","352 pages, Paperback","First published January 1, 1981","['Whitbread Award for First Novel (1981)', 'So...",A Good Man in Africa,NaN,['Morgan Leafy'],"352 pages, Paperback","January 14, 2003 by Vintage",9781400030026 (ISBN10: 1400030021),English,['Kinjanja']
2,It Had to Be You,Janice Thompson,0.00,Bella couldn't be happier that her long-feudin...,"Welcome to the wacky, wonderful world of inspi...","['Christian Fiction', 'Romance', 'Christian', ...",2152.0,132.0,"[14.0, 86.0, 421.0, 735.0, 896.0]","336 pages, Paperback","First published January 1, 2010",NaN,NaN,"['Weddings by Bella', 'Galveston Weddings']",NaN,"336 pages, Paperback","May 1, 2010 by Revell",9780800733445 (ISBN10: 0800733444),English,"['Galveston, Texas']"
3,Missing,Kelley Armstrong,6.99,Everyone knows they left. No one knows they're...,Kelley Armstrong has been telling stories sinc...,"['Young Adult', 'Mystery', 'Thriller', 'Fictio...",3393.0,491.0,"[54.0, 185.0, 799.0, 1308.0, 1047.0]","515 pages, ebook","First published April 18, 2017",NaN,Reeve's End (working title only),NaN,"['Edie Greene', 'Winter Crane', 'Tanner Robson...","515 pages, ebook","April 18, 2017 by Crown Books for Young Readers",9780399550348 (ISBN10: 0399550348),English,NaN
4,Dawn of Rebellion,Michelle Lynn,0.00,"My name is Dawn Nolan, and I've grown up on th...",Michelle moved around the Midwest most of her ...,"['Young Adult', 'Dystopia', 'Romance', 'Scienc...",142.0,76.0,"[1.0, 6.0, 23.0, 41.0, 71.0]","321 pages, Paperback","First published January 1, 2013",NaN,Dawn of Rebellion,['Dawn of Rebellion'],NaN,"321 pages, Paperback","October 17, 2013 by CreateSpace Independent Pu...",9781492942429 (ISBN10: 1492942421),English,NaN


In [14]:
df2 = pd.read_csv('./../../data/goodreads_data 2.csv')
df2.head(5)

,title,author,price,description,author_description,genres,n_ratings,n_reviews,ratings,pages_format,publication_info,literary_awards,original_title,series,characters,format,published,isbn,language,setting
0,The Weed That Strings the Hangman's Bag,Alan Bradley,4.99,From Dagger Award-winning and internationally ...,Librarian Note: There is more than one author ...,"['Mystery', 'Fiction', 'Historical Fiction', '...",55683.0,5755.0,"[394.0, 1541.0, 12234.0, 25708.0, 15806.0]","364 pages, Hardcover","First published March 9, 2010",['Goodreads Choice Award Nominee for Mystery &...,The Weed That Strings the Hangman's Bag,['Flavia de Luce '],"['Flavia de Luce', 'Ophelia de Luce', 'Daphne ...","364 pages, Hardcover","March 9, 2010 by Delacorte Press",9780385342315 (ISBN10: 0385342314),English,NaN
1,A Good Man in Africa,William Boyd,11.99,"In the small African republic of Kinjanja, Bri...","Note: William^^BoydOf Scottish descent, Boyd w...","['Fiction', 'Africa', 'Humor', 'Novels', 'Hist...",5015.0,298.0,"[90.0, 257.0, 1221.0, 2182.0, 1265.0]","352 pages, Paperback","First published January 1, 1981","['Whitbread Award for First Novel (1981)', 'So...",A Good Man in Africa,NaN,['Morgan Leafy'],"352 pages, Paperback","January 14, 2003 by Vintage",9781400030026 (ISBN10: 1400030021),English,['Kinjanja']
2,Missing,Kelley Armstrong,6.99,Everyone knows they left. No one knows they're...,Kelley Armstrong has been telling stories sinc...,"['Young Adult', 'Mystery', 'Thriller', 'Fictio...",3393.0,491.0,"[54.0, 185.0, 799.0, 1308.0, 1047.0]","515 pages, ebook","First published April 18, 2017",NaN,Reeve's End (working title only),NaN,"['Edie Greene', 'Winter Crane', 'Tanner Robson...","515 pages, ebook","April 18, 2017 by Crown Books for Young Readers",9780399550348 (ISBN10: 0399550348),English,NaN
3,Dawn of Rebellion,Michelle Lynn,0.00,"My name is Dawn Nolan, and I've grown up on th...",Michelle moved around the Midwest most of her ...,"['Young Adult', 'Dystopia', 'Romance', 'Scienc...",142.0,76.0,"[1.0, 6.0, 23.0, 41.0, 71.0]","321 pages, Paperback","First published January 1, 2013",NaN,Dawn of Rebellion,['Dawn of Rebellion'],NaN,"321 pages, Paperback","October 17, 2013 by CreateSpace Independent Pu...",9781492942429 (ISBN10: 1492942421),English,NaN
4,Smarter Than You Think: How Technology Is Chan...,Clive Thompson,11.99,A revelatory and timely look at how technology...,Clive Thompson is a Canadian freelance journal...,"['Nonfiction', 'Technology', 'Psychology', 'Sc...",2468.0,346.0,"[29.0, 116.0, 532.0, 1076.0, 715.0]","343 pages, Paperback","First published January 1, 2013",NaN,Smarter Than You Think: How Technology Is Chan...,NaN,NaN,"343 pages, Paperback","August 26, 2014 by Penguin Books",9780143125822 (ISBN10: 0143125826),English,NaN


In [15]:
df3 = pd.read_csv('./../../data/goodreads_data 3.csv')
df3.head(5)

,title,author,price,description,author_description,genres,n_ratings,n_reviews,ratings,pages_format,publication_info,literary_awards,original_title,series,characters,format,published,isbn,language,setting
0,The Weed That Strings the Hangman's Bag,Alan Bradley,4.99,From Dagger Award-winning and internationally ...,Librarian Note: There is more than one author ...,"['Mystery', 'Fiction', 'Historical Fiction', '...",55684.0,5756.0,"[394.0, 1540.0, 12236.0, 25708.0, 15806.0]","364 pages, Hardcover","First published March 9, 2010",['Goodreads Choice Award Nominee for Mystery &...,The Weed That Strings the Hangman's Bag,['Flavia de Luce '],"['Flavia de Luce', 'Ophelia de Luce', 'Daphne ...","364 pages, Hardcover","March 9, 2010 by Delacorte Press",9780385342315 (ISBN10: 0385342314),English,NaN
1,A Good Man in Africa,William Boyd,11.99,"In the small African republic of Kinjanja, Bri...","Note: William^^BoydOf Scottish descent, Boyd w...","['Fiction', 'Africa', 'Humor', 'Novels', 'Hist...",5017.0,298.0,"[90.0, 257.0, 1222.0, 2183.0, 1265.0]","352 pages, Paperback","First published January 1, 1981","['Whitbread Award for First Novel (1981)', 'So...",A Good Man in Africa,NaN,['Morgan Leafy'],"352 pages, Paperback","January 14, 2003 by Vintage",9781400030026 (ISBN10: 1400030021),English,['Kinjanja']
2,It Had to Be You,Janice Thompson,0.00,Bella couldn't be happier that her long-feudin...,"Welcome to the wacky, wonderful world of inspi...","['Christian Fiction', 'Romance', 'Christian', ...",2152.0,132.0,"[14.0, 86.0, 421.0, 735.0, 896.0]","336 pages, Paperback","First published January 1, 2010",NaN,NaN,"['Weddings by Bella', 'Galveston Weddings']",NaN,"336 pages, Paperback","May 1, 2010 by Revell",9780800733445 (ISBN10: 0800733444),English,"['Galveston, Texas']"
3,Missing,Kelley Armstrong,6.99,Everyone knows they left. No one knows they're...,Kelley Armstrong has been telling stories sinc...,"['Young Adult', 'Mystery', 'Thriller', 'Fictio...",3393.0,491.0,"[54.0, 185.0, 799.0, 1308.0, 1047.0]","515 pages, ebook","First published April 18, 2017",NaN,Reeve's End (working title only),NaN,"['Edie Greene', 'Winter Crane', 'Tanner Robson...","515 pages, ebook","April 18, 2017 by Crown Books for Young Readers",9780399550348 (ISBN10: 0399550348),English,NaN
4,Dawn of Rebellion,Michelle Lynn,0.00,"My name is Dawn Nolan, and I've grown up on th...",Michelle moved around the Midwest most of her ...,"['Young Adult', 'Dystopia', 'Romance', 'Scienc...",142.0,76.0,"[1.0, 6.0, 23.0, 41.0, 71.0]","321 pages, Paperback","First published January 1, 2013",NaN,Dawn of Rebellion,['Dawn of Rebellion'],NaN,"321 pages, Paperback","October 17, 2013 by CreateSpace Independent Pu...",9781492942429 (ISBN10: 1492942421),English,NaN
